In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 1. Importing data

I have chosen to copy paste the table data from Wikipedia in Excel and save it as csv. In this case, that apperas to me as quickest and straightforward way.

In [3]:
df=pd.read_csv('Toronto_Neighb.csv')
df.head()

,Postal code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,Moore Park / Summerhill East


## 2. Clean the dataframe

I removed allo the rows with Not assigned Borough. In the Wikipedia page the neighborhoods are already grouped by postal code (probabily newer version compared to when the assignment was created), so I only had to replace "/" with "," as separator for neighborhoods with same postal codes.

In [4]:
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)

df['Neighborhood'] = df['Neighborhood'].str.replace('/',',')
df.head()

,Postal code,Borough,Neighborhood
0,M4N,Central Toronto,Lawrence Park
1,M4P,Central Toronto,Davisville North
2,M4R,Central Toronto,North Toronto West
3,M4S,Central Toronto,Davisville
4,M4T,Central Toronto,"Moore Park , Summerhill East"


In [5]:
df.shape

(103, 3)

## 2. Check if the cretaed dataframe corresponds to the picture in the assignment 

In [6]:
pc = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
li=[]
for i in pc:
  df1=df.loc[df['Postal code']==i]
  df1.index.name = None
  li.append(df1)
df_test=pd.concat(li)
df_test.reset_index(drop=True, inplace=True)
df_test

,Postal code,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill , Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford , Maryvale"
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ..."


## 3. Read in the coordinates file

In [80]:
dfg=pd.read_csv('Geospatial_Coordinates.csv')
dfg.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 4. Merge coordinates with neighborhoods dataframe

In [81]:
dfg1=dfg.rename(columns={'Postal Code':'Postal code'})
dfg1.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [92]:
df1 = df.merge(dfg1, on="Postal code", how="left")
df1.head()


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park , Summerhill East",43.689574,-79.383160


## 5. Check against the assignment picture

In [97]:
lis=[]
for i in pc:
  df1_=df1.loc[df1['Postal code']==i]
  df1_.index.name = None
  lis.append(df1_)
df1_test=pd.concat(lis)
df1_test.reset_index(drop=True, inplace=True)
df1_test


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Wexford , Maryvale",43.750072,-79.295849
7,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower , King and Spadina , Railway Lands , ...",43.628947,-79.394420
